In [ ]:
from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
import networkx as nx

In [ ]:
# implementez votre solution ici...
def noeud(h,i,j,q):
  #algo de Dijkstra, recursif
  #h: matrice de distances geodesiques a completer: a chauqe ligne est un element du graph 
  #i: le point du graphe dont on cherche a calculer les distances geodisiques vis a vis des autres points
  #j: le point auquel l,algorythme est rendu
  #q: matrice des arretes restantes: inf lorsque l,arrete a deja ete visitee.
  #Remarques: avec dijkstra, on n,a pas besoin de revisiter une arrete (a chaque passage de noeud, q est plus inf)
  # l,algo est assez optimal pour un seul pts i, mais suppose que l,utilisateur veuille potentiellement executer
  # sur des pts differents (comme tous les pts de h), alors il serait fortement profitable de permettre en sortie une matrice
  # des "chemins les plus courts", car si le plus court chemin entre i et i3 passe par i2, alors le plus court chemin entre i2 et i3
  # est deja connu.
  #NV:=non visite
  # print(h)
  arr=np.where(np.isfinite(q[j,:]))[0]#arr: les arretes
  # print(arr)
  for c in arr:
    t=h[i,j]+q[j,c]
    if t<h[i,c] :
      h[i,c]=t
  # for c in arr:
  #   h,q=noeud(h,i,c,q)#on passe au noeud adjacent NV, equivalent a une arrete NV le reliant a j
  return h# arr pour connaitre les 

##contient une erreur apparement
def etape(h,i,arr,nodes):
  #h contient les distances calculees a l,etape consideree
  #arr contient les arretes (n x n, inf lorsque pas d,arrete ou sur diag)
  #node marque la progression de l,algo:
  # -inf pour noeuds encore non visites
  # -1 pour noeuds a regarder
  # -0 pour noeuds elimines
  #i est le noeud dont on calcule les distances
  #Retourne:
  #h', une matrice de distance corrigee
  #newnode nouveau qui marque la prochaine etape de l,algo
  #l,algo se termine lorsque newnode ne contient plus de 1
  indices=np.where(nodes==1)[0]
  newNodes=[]#liste des noeuds connectes aux noeuds de l,etape
  for j in indices:#pour chaque noeud de l,etape,
    h =noeud(h,i,j,arr)#on voit si on peut ameliorer les distances de h
    a=np.where(np.isfinite(arr[j,:]))[0]
    newNodes.append(a)
  
  nodes=np.where(nodes==1,0,nodes)#on supprime les noeuds regardes
  newNodes=np.unique(newNodes)#tous les noeuds connectes aux noeuds visites
  # newNodes=np.where( (np.logical_and(np.isinf(nodes),np.where(nodes)[0] in newNodes)),1,nodes)#ERREUR: histoire de .any()
  newNodes= np.intersect1d(np.where(np.isinf(nodes),newNodes))
  return h,newNodes

###Erreur dans l'algo: On ne peut pas proceder noeud par noeud: il faut commencer par les oeuds avec moins d'arretes les separant 
###du noeud sur lequel on applique Dijkstra
def EDijkstra(h,i):
  #trouve les distances geodesiques d,un pnts
  #h: graphe ou matrice de distance partielle: inf lrsq pnts pas d'arrete, 0 pr le noeud
  #i: pnts (ligne de h) pour lequel l,on calcule les distances sur graphe.
  #Sorties: 
  # matrice h avec rangee i completee autant que possible: h peut etre non connectee
  # if ( (not (np.diag(h)==0).all()) or (not (np.transpose(h)==h).all())): 
  #   return print('Ceci n,est pas une matrice de distances, np.inf est ok, mais exige symetric et diag=0')
  # #q represente les arretes qui n,ont pas ete visitees; pour simplifier le code,
  #on garde le format symetrique: q[i,j] est la meme arrete que q[j,i], bien que l,on ne repasse
  #pas deux fois sur la meme arrete
  q=h.copy()
  np.fill_diagonal(q,np.inf)#les arretes sont les valeurs non inf
  # print('depart')
  # print(h)
  solution,residus =noeud(h,i,i,q)#algo recursif
  # print(residus)
  return solution

def geodesique(h,pnts):
  #h est une matrice de distances geodesiques incomplete
  #calcule les distances geodesiques pour les i points
  #if (not ( np.all(np.transpose(h)==h) and np.all(np.diag(h)==0)):
  #  h=cdist(h,h,'euclidean')
  n=np.shape(h)[0]
  arr=h.copy()
  np.fill_diagonal(arr,np.inf)#les arretes sont les valeurs non inf
  for i in pnts:#on calcule les distances geodesiques
    nodes=np.ones(n)*np.inf
    nodes[i]=1 #on part l,algo au noeud i
    while (1 in nodes): h,nodes=etape(h,i,arr,nodes)
  return h


def isomap(data,epsF,newDim):
  #function epsF pour determiner le eps d'après matrice de distances
  #newDim: dimension de sortie (projection MDS sur newDim de vecteurs propres)
    ndata = np.shape(data)[0]
    ndim = np.shape(data)[1]
    d = cdist(data,data,'euclidean')
    eps = epsF(d)
    neighbours = np.where(d<eps,d,np.inf)
    #h:matrice de distances, infinies sauf neighbours, ajustement par dijkstra
    h=geodesique(neighbours,range(ndata))
    return h#MDS(h,newDim)